# IBM Capstone Project
### by Marc Gou

## Phase 2: Segmenting and Clustering Neighborhoods in Toronto

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

1. Start by creating a new Notebook for this assignment.
---
2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:
![alt text][logo]
[logo]: https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/7JXaz3NNEeiMwApe4i-fLg_40e690ae0e927abda2d4bde7d94ed133_Screen-Shot-2018-06-18-at-7.17.57-PM.png?expiry=1546646400000&hmac=CRqh9ZyRbEZRxrrEhJftbfRQumTwfQDD3wnfdlYgc08
---  
3. To create the above dataframe:

  * The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
  * Only process the cells that have an assigned borough. Ignore cells with a borough that is **Not assigned**.
  * More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that **M5A** is listed twice and has two neighborhoods: **Harbourfront** and **Regent Park**. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in **row 11** in the above table.
  * If a cell has a borough but a **Not assigned** neighborhood, then the neighborhood will be the same as the borough. So for the **9th** cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be **Queen's Park.**
  * Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
  * In the last cell of your notebook, use the **.shape** method to print the number of rows of your dataframe.
---    
4. Submit a link to your Notebook on your Github repository. **(10 marks)**

## My Submission:

**Step 1: Installation and import of useful libraries**

In [87]:
# Installation of useful libraries
# !conda install -c conda-forge beautifulsoup4 --yes

# Import of useful libraries
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request as req

**Step 2: Request to the wiki page and parse it with BeautifulSoup, and locate the Toronto neighborhoods table**

In [88]:
# Open the URL
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_req = req.urlopen(wiki_url)

# Parsing with BeautifulSoup
wiki_soup = BeautifulSoup(wiki_req, 'lxml')
# print(wiki_soup.prettify())

*We can see that the table of Toronto neighborhoods are under the class "wikitable sortable"*

In [89]:
# Locate the table containing the 
neig_table = wiki_soup.find('table', attrs={'class':'wikitable sortable'})

**Step 3: Convert the XML Tree above to a dataframe**

*We can see that the headers are marked with the tag __th__*

In [90]:
# Extract all the headers
table_h = neig_table.findAll('th')

# Define the headers
headers = []
for h in table_h:
    headers.append(h.contents[0].replace('\n',''))

*We can see that each neighborhood are marked with the tag __tr__ with __td__ for the value of their attributes*

In [91]:
# Extract all the neighborhood data
table_n = neig_table.findAll('td')

# Insert the data, 3 elements for each postal code
full_data = []
for i in range(0, len(table_n)-3, 3):
    pc_data = []
    e1 = table_n[i].contents[0]
    e2 = table_n[i+1].contents[0]
    e3 = table_n[i+2].contents[0]
    pc_data.append(e1.replace('\n',''))
    if(str(e2)[0:2] == '<a'):
        pc_data.append(e2.get('title'))
    else:
        pc_data.append(e2.replace('\n',''))    
    if(str(e3)[0:2] == '<a'):
        pc_data.append(e3.get('title'))
    else:
        pc_data.append(e3.replace('\n',''))
    full_data.append(pc_data)

*Now let's convert the data to a dataframe*

In [92]:
df = pd.DataFrame(data = full_data, columns = headers)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront (Toronto)


**Step 4: Process the dataframe according to the requirements**

In [93]:
# Ignore cells with a borough that is Not assigned.
df = df[df.Borough != 'Not assigned'].reset_index(drop=True)

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df.Borough

# When more than one neighborhood can exist in one postal code area: these two rows will be combined into one row 
# with the neighborhoods separated with a comma as shown in row 11 in the above table.
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x)).reset_index()
df

,Postcode,Borough,Neighbourhood
0,M1B,"Scarborough, Toronto","Rouge, Toronto, Malvern, Toronto"
1,M1C,"Scarborough, Toronto","Highland Creek (Toronto), Rouge Hill, Port Uni..."
2,M1E,"Scarborough, Toronto","Guildwood, Morningside, Toronto, West Hill, To..."
3,M1G,"Scarborough, Toronto","Woburn, Toronto"
4,M1H,"Scarborough, Toronto","Woburn, Toronto"
5,M1J,"Scarborough, Toronto",Scarborough Village
6,M1K,"Scarborough, Toronto","East Birchmount Park, Ionview, Kennedy Park, T..."
7,M1L,"Scarborough, Toronto","Clairlea, Golden Mile, Toronto, Oakridge, Toronto"
8,M1M,"Scarborough, Toronto","Cliffcrest, Cliffside, Toronto, Scarborough Vi..."
9,M1N,"Scarborough, Toronto","Birch Cliff, Cliffside West"


*In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe*

In [94]:
df.shape

(103, 3)

**Step 5: Process the dataframe according to the requirements**

*Download the geospatial data, save it as a CSV file and read it into a pandas dataframe*

In [95]:
!wget -q -O 'toronto_geodata.csv' http://cocl.us/Geospatial_data
geo_df = pd.read_csv('toronto_geodata.csv')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [98]:
toronto_merged = df
toronto_merged = toronto_merged.join(geo_df.set_index('Postal Code'), on='Postcode')
toronto_merged

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,"Scarborough, Toronto","Rouge, Toronto, Malvern, Toronto",43.806686,-79.194353
1,M1C,"Scarborough, Toronto","Highland Creek (Toronto), Rouge Hill, Port Uni...",43.784535,-79.160497
2,M1E,"Scarborough, Toronto","Guildwood, Morningside, Toronto, West Hill, To...",43.763573,-79.188711
3,M1G,"Scarborough, Toronto","Woburn, Toronto",43.770992,-79.216917
4,M1H,"Scarborough, Toronto","Woburn, Toronto",43.773136,-79.239476
5,M1J,"Scarborough, Toronto",Scarborough Village,43.744734,-79.239476
6,M1K,"Scarborough, Toronto","East Birchmount Park, Ionview, Kennedy Park, T...",43.727929,-79.262029
7,M1L,"Scarborough, Toronto","Clairlea, Golden Mile, Toronto, Oakridge, Toronto",43.711112,-79.284577
8,M1M,"Scarborough, Toronto","Cliffcrest, Cliffside, Toronto, Scarborough Vi...",43.716316,-79.239476
9,M1N,"Scarborough, Toronto","Birch Cliff, Cliffside West",43.692657,-79.264848
